In [4]:
import CONST
import wandb
from pprint import pprint

from entities import WoundDataset
from models.trainer.diffuser import Diffuser

In [ ]:
# Login wandb
wandb_key = "a8b5a7676a58d9b5b1e686fd9d349bc25f18d07c"
wand_logged = wandb.login(key=wandb_key)

In [5]:
# Define sweep configuration
sweep_config = {
    "method": "bayes",
    "metric": {
        "name": "train_mse_loss",
        "goal": "minimize",
    },
    "parameters": {
        "learning_rate": {"distribution": "uniform", "min": 5e-5, "max": 3e-4},
        "noise_steps": {"values": [50, 100, 500, 1000]},
        "variance_schedule_type": {"values": ["linear", "quadratic", "sigmoid", "cosine"]},
        "attn_heads": {"values": [1, 2, 4]}
    }
}

pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'train_mse_loss'},
 'parameters': {'attn_heads': {'values': [1, 2, 4]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.0003,
                                  'min': 5e-05},
                'noise_steps': {'values': [50, 100, 500, 1000]},
                'variance_schedule_type': {'values': ['linear',
                                                      'quadratic',
                                                      'sigmoid',
                                                      'cosine']}}}


In [6]:
# Initialise sweep controller on WandB server
# After initialised, we do not need to run this again
# sweep_id = wandb.sweep(sweep_config, project="DDPM_hyper_tuning")
# pprint(sweep_id)

Create sweep with ID: s4cs0c7m
Sweep URL: https://wandb.ai/qhuy0168/DDPM_hyper_tuning/sweeps/s4cs0c7m
's4cs0c7m'


In [8]:
# Define the agent
# Run this on agent computers
def train():
    """
    Training trigger for hyper-parameter optimisation
    :return: None
    """
    # Init the run
    run = wandb.init(project="DDPM_hyper_tuning")

    # Get the configuration for instance
    config = wandb.config

    # Similar approach to full train
    # The data
    dataset = WoundDataset(
        image_dir=CONST.PROCESSED_IMAGES_DIR,
        segment_dir=CONST.PROCESSED_SEGMENT_DIR,
        target_tensor_size=CONST.DIFFUSER_SETTINGS.INPUT_SIZE,
        embedding_dir=CONST.PROCESSED_EMBEDDING_DIR
    )

    diffuser = Diffuser(
        dataset=dataset,
        batch_size=8,
        num_workers=2,
        epochs=1000,
        run_name=CONST.DIFFUSER_SETTINGS.RUN_NAME,
        output_dir=CONST.DIFFUSER_SETTINGS.OUTPUT_DIR,
        max_lr=config.learning_rate,
        noise_steps=config.noise_steps,
        variance_schedule_type=config.variance_schedule_type,
        attn_heads=config.attn_heads,
        wandb_run=run,
    )

    diffuser.fit()

In [ ]:
# Start the sweep agent
wandb.agent("s4cs0c7m", function=train, count=10)